### Camadas de Convolução no PyTorch

As camadas de convolução são componentes fundamentais das redes neurais convolucionais (CNNs), amplamente utilizadas para processamento de imagens, análise de vídeo, e aplicações de reconhecimento de padrões. No PyTorch, estas camadas são implementadas através do módulo `torch.nn`, que oferece várias classes para diferentes tipos de convolução.

#### Importância nas CNNs

As camadas de convolução são essenciais para capturar hierarquias de características visuais — dos detalhes mais simples nos primeiros níveis até padrões complexos e abstratos nas camadas mais profundas. Estas características tornam as CNNs extremamente eficazes para tarefas que dependem da compreensão de contextos visuais e da identificação de objetos em imagens e vídeos.

As camadas de convolução no PyTorch são otimizadas para oferecer alto desempenho e flexibilidade, permitindo aos desenvolvedores e pesquisadores construir arquiteturas de CNNs avançadas e eficientes para uma ampla gama de aplicações.

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

# Sinal
x = torch.tensor([5, 4, 8, 7, 9, 3, 6], dtype=torch.float32)
# Filtro
weight = torch.tensor([1, 2, 3], dtype=torch.float32)
# Tamanho do filtro
ks = len(weight)

# Redimensiona o sinal para o tamanho 1x1xlen(x). Ou seja, um batch contendo um único sinal, 
# e esse sinal possui um único canal
x = x.reshape(1,1,len(x))
# Redimensiona o filtro. O primeiro valor 1 possui um significado diferente do que
# no caso do sinal. Depois será explicado.
weight = weight.reshape(1,1,len(weight))
# Realiza a convolução
y = F.conv1d(x, weight)
y.shape

torch.Size([1, 1, 5])

O tamanho da entrada é 7 e da saída é 5. Isso porque o Pytorch realiza a convolução apenas nas posições que não necessitam de preenchimento de borda. Mas modificar o tamanho do resultado é indesejável. 

É muito comum realizarmos a convolução com padding para manter o tamanho do sinal:

In [2]:
# padding = ks//2 garante que a saída sempre terá o mesmo tamanho que a entrada
y = F.conv1d(x, weight, padding=ks//2)
print(y.shape)

torch.Size([1, 1, 7])


Dado o sinal [5, 4, 8, 7, 9, 3, 6] e filtro [1,2,3], nossa saída deve ser:

* y[0] = 1\*0 + 2\*5 + 3\*4 = 22
* y[1] = 1\*5 + 2\*4 + 3\*8 = 37
* ...
* y[6] = 1\*3 + 2\*6 + 3\*0 = 15

Note que a função realiza a correlação-cruzada, e não a convolução. Mas para redes neurais isso não importa.

In [3]:
print(y)

tensor([[[22., 37., 41., 49., 34., 33., 15.]]])


Em redes neurais a convolução possui o conceito de bias, que é simplesmente um valor constante que é adicionado ao resultado:

In [4]:
bias = torch.tensor([5.])
# Adiciona o valor 5 a cada elemento do resultado da convolução
y_bias = F.conv1d(x, weight, padding=ks//2, bias=bias)
print(y_bias)
print(torch.allclose(y+bias, y_bias))

tensor([[[27., 42., 46., 54., 39., 38., 20.]]])
True


### Camada de convolução

Uma camada de convolução aplica um conjunto de filtros aprendíveis aos dados de entrada. Cada filtro em uma camada de convolução possui pesos que são aprendidos durante o treinamento do modelo. A operação de convolução envolve a multiplicação desses pesos pelos valores nas regiões da entrada, seguida pela soma dos produtos para produzir um mapa de características (feature map). Este mapa representa informações espaciais que são essenciais para tarefas de visão computacional.

#### Tipos de Camadas de Convolução

- **`nn.Conv1d`**: Aplicada em dados sequenciais, como séries temporais ou texto (quando os dados são tratados como uma sequência de tokens ou caracteres), operando ao longo de uma única dimensão espacial.

- **`nn.Conv2d`**: Esta é a camada de convolução bidimensional mais comumente usada para processar imagens. Ela aplica filtros que deslizam sobre a largura e altura da imagem de entrada para produzir os mapas de características.

- **`nn.Conv3d`**: Usada para dados volumétricos ou vídeos, onde a convolução é aplicada em três dimensões — profundidade, altura e largura.

#### Parâmetros Comuns em Métodos de Convolução do PyTorch

1. **in_channels**:
   - **Descrição**: Número de canais na imagem ou tensor de entrada. Por exemplo, imagens RGB têm 3 canais.
   - **Tipo**: `int`

2. **out_channels**:
   - **Descrição**: Número de filtros que a camada de convolução vai aplicar. Este número também determina o número de canais na saída, pois cada filtro produz um mapa de características único.
   - **Tipo**: `int`

3. **kernel_size**:
   - **Descrição**: Tamanho do filtro ou kernel usado na convolução. Pode ser especificado como um único número (para um tamanho de kernel uniforme em todas as dimensões) ou como uma tupla que define o tamanho do kernel para cada dimensão.
   - **Tipo**: `int` ou `tuple`

4. **stride**:
   - **Descrição**: O número de passos que o filtro se move ao longo de cada dimensão da entrada. Um `stride` maior resulta em uma redução proporcional das dimensões do mapa de características de saída. Similar ao `kernel_size`, pode ser um único número ou uma tupla.
   - **Tipo**: `int` ou `tuple`

5. **padding**:
   - **Descrição**: Adiciona bordas de zeros em torno da entrada para permitir que a convolução seja aplicada às bordas da imagem de entrada, mantendo o tamanho da saída. Pode ser especificado como um único número (padding uniforme em todas as bordas) ou como uma tupla para definir padding específico por dimensão.
   - **Tipo**: `int` ou `tuple`

6. **dilation**:
   - **Descrição**: Especifica o espaçamento entre os elementos do kernel. Um `dilation` maior resulta em um "filtro dilatado", útil para capturar características em escalas maiores sem aumentar o número de parâmetros. Funciona efetivamente aumentando o campo receptivo do filtro.
   - **Tipo**: `int` ou `tuple`

7. **groups**:
   - **Descrição**: Controla a conexão entre as entradas e as saídas. Por padrão, `groups=1`, o que significa que cada filtro é aplicado a todos os canais de entrada. Se `groups` é igual ao número de canais de entrada, isso resulta numa convolução chamada "depthwise". Isso permite que a operação seja separada em grupos de canais, cada um realizando uma convolução independente.
   - **Tipo**: `int`

8. **bias**:
   - **Descrição**: Se `True`, adiciona um termo de viés à saída de cada mapa de características. Normalmente deixado como `True` a menos que a próxima camada imediatamente aplique uma normalização que torne o termo de viés redundante (como batch normalization).
   - **Tipo**: `bool`

In [5]:
# A entrada terá 1 canal, queremos apenas 1 canal de saída. O tamanho do filtro é ks, o 
# padding é metade do tamanho do filtro e a camada não terá bias
conv = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=ks, padding=ks//2, bias=False)
y = conv(x)
print(y)

tensor([[[-0.1076, -0.2768, -1.5261, -2.0427, -4.5097, -2.2766, -3.0912]]],
       grad_fn=<ConvolutionBackward0>)


Uma camada de convolução consiste em um filtro possuindo valores aleatórios. Esse filtro possui o parâmetro requires_grad=True por padrão. Podemos alterar os valores do filtro se quisermos:

In [6]:
print(conv.weight.shape)
print(conv.weight)

with torch.no_grad():
    conv.weight[:] = weight

# Mesmo resultado que a convolução que fizemos antes:
conv(x)

torch.Size([1, 1, 3])
Parameter containing:
tensor([[[-0.3941, -0.3182,  0.3708]]], requires_grad=True)


tensor([[[22., 37., 41., 49., 34., 33., 15.]]], grad_fn=<ConvolutionBackward0>)

### Relação entre convolução e combinação linear

Uma convolução nada mais é do que uma combinação linear com menos parâmetros

In [7]:
# Matriz de convolução. Cada linha representa uma posição do kernel. Por exemplo,
# a linha 0 fará a operação 2*x[0]+3*x[1]+0*x[2]+...
matrix = torch.tensor([[2, 3, 0, 0, 0, 0, 0],
                       [1, 2, 3, 0, 0, 0, 0],
                       [0, 1, 2, 3, 0, 0, 0],
                       [0, 0, 1, 2, 3, 0, 0],
                       [0, 0, 0, 1, 2, 3, 0],
                       [0, 0, 0, 0, 1, 2, 3],
                       [0, 0, 0, 0, 0, 1, 2]], dtype=torch.float32)

F.linear(x, matrix)

tensor([[[22., 37., 41., 49., 34., 33., 15.]]])

Acima temos uma matrix 7x7 que recebe 7 atributos de entrada e gera 7 atributos de saída. Mas a combinação linear dos atributos de entrada sempre envolvem apenas 3 parâmetros.

### Convolução com mais de um canal

1. **Entrada da Camada**: 
   - A entrada para a camada de convolução consiste em um conjunto (batch) de imagens. Por exemplo, se temos um batch com 2 imagens, e cada imagem tem múltiplos canais (como as três cores RGB) e um tamanho específico (altura H e largura W), a entrada é representada por um tensor de quatro dimensões: N (número de imagens no batch) × Cin (número de canais por imagem) × H × W.

2. **Filtros da Camada**:
   - A camada de convolução utiliza vários filtros (ou kernels) para processar a entrada. Cada filtro é capaz de extrair características específicas da imagem, como bordas, texturas ou outros padrões. Se a camada possui Cout filtros, cada um ajustado para analisar todos os canais de entrada, a estrutura de cada filtro será Cin × KW × KH, onde KW e KH são a largura e altura do filtro, respectivamente.
   - Todos os filtros são armazenados em um tensor de tamanho Cout × Cin × KW × KH. Além disso, há um valor de bias para cada filtro, armazenado em um tensor de tamanho Cout.

3. **Operação de Convolução**:
   - Durante a convolução, cada filtro "desliza" sobre a imagem inteira, aplicando-se a cada posição possível da imagem. Em cada posição, o filtro realiza um cálculo que envolve multiplicar seus valores pelos valores correspondentes na imagem e somar todos esses produtos.
   - O resultado dessa operação em cada posição é então somado com o valor de bias correspondente ao filtro.

4. **Saída da Camada**:
   - A saída da camada de convolução é um novo conjunto de imagens (um batch), onde cada imagem agora possui Cout canais, um para cada filtro aplicado. O tamanho de cada imagem de saída (H' × W') depende do tamanho original da imagem, do tamanho do filtro, da quantidade de padding aplicada (adicionando zeros ao redor da imagem para permitir que o filtro se aplique nas bordas) e do stride (quantos pixels o filtro pula após cada aplicação).
   - Comumente, o número de canais de saída (Cout) é bastante alto, como 64, 128 ou 256, permitindo que a rede capture uma ampla gama de características.

![Camada de Convolução](./conv_layers.png)

In [8]:
# Batch contendo um sinal de tamanho 7 e com 4 canais
x = torch.rand(size=(1,4,7))
# Camada que recebe sinal com 4 canais e gera um sinal com 5 canais.
conv = nn.Conv1d(in_channels=4, out_channels=5, kernel_size=ks, padding=ks//2, bias=False)

y = conv(x)
print('x\n',x)
# Saída possui tamanho 1x5x7
print('y\n',y)

x
 tensor([[[0.4016, 0.1977, 0.0868, 0.5421, 0.9579, 0.4345, 0.9196],
         [0.3162, 0.1147, 0.3042, 0.3195, 0.3976, 0.4138, 0.6200],
         [0.3600, 0.6513, 0.9571, 0.4475, 0.5283, 0.7149, 0.4173],
         [0.6390, 0.4250, 0.2405, 0.4718, 0.2524, 0.7026, 0.4903]]])
y
 tensor([[[-0.0387,  0.1202,  0.0074, -0.2349, -0.3178, -0.4426, -0.1302],
         [-0.1361, -0.1036, -0.1337, -0.0944, -0.0503, -0.2261,  0.0394],
         [ 0.2013, -0.0668, -0.2274, -0.1265,  0.1902,  0.0051, -0.0249],
         [ 0.0797,  0.3654,  0.1256,  0.2440,  0.2098,  0.1537,  0.3551],
         [ 0.2287,  0.1897,  0.0727,  0.0493,  0.2980,  0.1749,  0.1166]]],
       grad_fn=<ConvolutionBackward0>)


* O **número de canais de saída** define o **número de filtros** que serão utilizados na camada de convolução. No nosso caso, temos 5 filtros
* Cada filtro possui **tamanho espacial ks**
* Cada filtro possui **profundidade 4**, pois o sinal de entrada possui 4 canais.
* Portanto, temos 5 filtros de tamanho 4 x ks cada
* Portanto, o tamanho do tensor .weight da camada de convolução possui tamanho 5 x 4 x ks

In [9]:
conv.weight.shape

torch.Size([5, 4, 3])

In [10]:
# Filtro 1 da camada
filtro1 = conv.weight[0]
# Região do sinal que corresponde quando o filtro está na posição 1
regiao = x[0,:,0:3]
# Resultado da convolução para esse ponto específico
res = (filtro1*regiao).sum()
# comparação do resultado
print(torch.allclose(y[0,0,1], res))

True


### Outros parâmetros da convolução

#### Stride

O stride refere-se ao `número de pixels que o filtro se desloca` ao longo das dimensões de entrada após cada operação de convolução. Em termos simples, o stride controla o quão longe o filtro se move cada vez que é aplicado durante a convolução. Um stride maior resulta em uma redução das dimensões do mapa de características produzido, já que o filtro é aplicado menos vezes sobre a entrada. Por exemplo, um stride de 2 significa que o filtro pula um pixel (ou unidade de medida em outras dimensões de dados) para cada passo, reduzindo aproximadamente pela metade o tamanho do mapa de características em comparação com um stride de 1, assumindo outros parâmetros constantes.

In [11]:
x = torch.tensor([5, 4, 8, 7, 9, 3, 6], dtype=torch.float32)
weight = torch.tensor([1, 2, 3], dtype=torch.float32)
x = x.reshape(1,1,len(x))
weight = weight.reshape(1,1,len(weight))

# Faz o filtro deslocar duas posições ao invés de deslocar uma posição por vez
y = F.conv1d(x, weight, stride=2)
print(y)

tensor([[[37., 49., 33.]]])


In [12]:
# O resultado de stride=2 é equivalente a indexar a saída de stride=1 pulando 2 índices
yt = F.conv1d(x, weight, stride=1)
print(torch.allclose(y, yt[0,0,::2]))

True


#### Dilatação

A dilatação, por sua vez, envolve a `inserção de espaços entre os pixels ou unidades dentro do filtro` usado na operação de convolução. Este processo aumenta efetivamente o tamanho ou a área do filtro sem adicionar novos pesos ao modelo. A dilatação permite que o filtro abranja uma área maior da entrada com o mesmo número de parâmetros, aumentando o campo receptivo do filtro. Por exemplo, uma dilatação de 2 significa que há um espaço de um pixel entre cada unidade do filtro, tornando o filtro espacialmente maior. Isso é especialmente útil para capturar informações em escalas maiores sem aumentar a complexidade computacional do modelo, já que o número de parâmetros permanece o mesmo.

In [13]:
y = F.conv1d(x, weight, dilation=2)
print(y)

tensor([[[48., 27., 44.]]])


In [14]:
# dilatação=2 é equivalente a inserir 0 entre os valores do filtro
wt = torch.tensor([1, 0, 2, 0, 3], dtype=torch.float32).reshape(1,1,-1)
yt = F.conv1d(x, wt, dilation=1)
print(torch.allclose(y, yt))

True


#### Impacto do Stride e da Dilatação

O uso de stride e dilatação afeta diretamente a forma como as camadas de convolução percebem e processam as entradas. Um stride maior pode acelerar o processamento e reduzir a resolução espacial dos mapas de características, útil para aumentar a abstração e reduzir o consumo de memória em redes profundas. A dilatação, por outro lado, permite que o modelo capture contextos mais amplos ou padrões de maior escala sem perder a resolução espacial ou aumentar o número de parâmetros, mantendo a eficiência computacional. Ambos os parâmetros são ajustáveis no PyTorch e podem ser configurados de acordo com as necessidades específicas da tarefa de aprendizado de máquina sendo abordada.






